In [ ]:
!pip install beautifulsoup4

from bs4 import BeautifulSoup
import csv
import requests
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def getdata(url):
  header = {'User-agent' :'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62'}
  r = requests.get(url, headers = header)
  return r.text

In [ ]:
url = 'https://old.reddit.com/r/todayilearned/top/?sort=top&t=all'

htmldata = getdata(url)
soup = BeautifulSoup(htmldata, 'html.parser')

In [ ]:
attrs = {'class': 'thing', 'data-whitelist-status': 'all_ads'}

counter = 1

while True:
  for post in soup.find_all('div', attrs=attrs):
    title = post.find('p', class_="title").text
    try:
      author = post.find('a', class_='author').text
    except:
      author = "deleted user"
    post_time = post.find('time').text
    comments = post.find('a', class_='comments').text.split()[0]
    if comments == "comment":
      comments = 0
    likes = post.find("div", attrs={"class": "score likes"}).text
    source = post.find('a', href=True)
    source_link = source['href']
    
    #Write to csv
    post_line = [counter, title, author, post_time, comments, likes, source_link]
    with open('/content/drive/MyDrive/Colab Notebooks/todayilearned_scrapped_data.csv', 'a') as f:
      writer = csv.writer(f)
      writer.writerow(post_line)
    
    counter += 1
    if counter > 950:
      break
  
  if counter > 950:
    break

  #Move to next page
  next_button = soup.find("span", class_="next-button")
  next_page_link = next_button.find("a").attrs['href']
  time.sleep(2)
  page = getdata(next_page_link)
  soup = BeautifulSoup(page, 'html.parser')